![title](https://i.ytimg.com/vi/Cgu5y-6VWfo/maxresdefault.jpg)

# Une étude de cas d’intelligence de localisation pour compare locations entre Paris et Lyon

 <center> <a href="http://www.roqueleal.me">Roque Leal</a> </center>

Déménager dans une autre ville est toujours excitant mais implique également de nombreuses décisions. L'une des plus grandes décisions est: où dois-je vivre?
Le quartier dans lequel vous vivez peut avoir un impact considérable sur votre première impression de la nouvelle ville, sans oublier que c'est une décision que vous DEVEZ prendre avant de déménager!
Certaines personnes vivaient dans des endroits très fréquentés et aimeraient changer de rythme lorsqu'elles bougent. D'autres voudraient la même énergie qu'ils vivaient dans leur vieille ville! Ce projet vise à comparer certains quartiers entre deux villes (dans ce cas, Chicago et Seattle) et à voir quelles zones sont similaires et lesquelles pourraient être complètement opposées.

Après quelques recherches de quartier, nous comparerons le quartier à déménager dans Paris, Lyon oú Marselle.

Les données que nous utiliserons seront des données de lieu de Foursquare. Les lieux que nous rassemblerons concerneront de nombreuses choses qu'une personne rechercherait lorsqu'elle vivrait, par exemple:
    - Épiceries
    - Abordabilité de ces lieux
    - Restaurants
    - Centres commerciaux
...et plus!
  
Voyons voir si nous pouvons les aider à décider.

In [5]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

from geopy.geocoders import Nominatim # latitude/longitude converter

import requests
from pandas.io.json import json_normalize #json files to pandas dataframe

import matplotlib.cm as cm
import matplotlib.colors as colors

import folium

print('Bibliothèques importées.')

Bibliothèques importées.


### Configurer l'API´s

In [1]:
CLIENT_ID = 'xxxxxx'
CLIENT_SECRET = 'xxxxxx'
VERSION = 'xxxxxx'

print('Client ID: ' + CLIENT_ID)
print('Client Secret: ' + CLIENT_SECRET)
print('Version: '+ VERSION)

Client ID: xxxxxx
Client Secret: xxxxxx
Version: xxxxxx


# Méthodologie

### Adresse Principale: importez à Paris les données de l'API Foursquare et créez DataFrame

In [6]:
c_address = '13 Rue de Poissy, 75005 Paris, Francia'

c_geolocator = Nominatim(user_agent='foursquare_agent')
c_location = c_geolocator.geocode(c_address)
c_latitude = c_location.latitude
c_longitude = c_location.longitude
print('13 Rue de Poissy, 75005 Paris, Francia, coordinates: ', c_latitude, c_longitude)

13 Rue de Poissy, 75005 Paris, Francia, coordinates:  48.8489808 2.3525466


In [8]:
radius = 500
LIMIT = 100000

c_url = 'xxxxxx'.format(
        CLIENT_ID,
        CLIENT_SECRET,
        c_latitude,
        c_longitude,
        VERSION,
        radius,
        LIMIT
        )

print(c_url)

xxxxxx


In [95]:
results = requests.get(c_url).json()

In [96]:
def get_category_type(row):
    try:
        category_list = row['categories']
    except:
        category_list = row['venue.categories']
    
    if len(category_list) == 0:
        return None
    else:
        return category_list[0]['name']

In [97]:
venues = results['response']['groups'][0]['items']

nearby_venues = json_normalize(venues) #flattens JSON

filtered_columns = ['venue.name', 'venue.id', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis = 1)

In [98]:
nearby_venues.head(10)

,venue.name,venue.id,venue.categories,venue.location.lat,venue.location.lng
0,La Tour d'Argent,4adcda11f964a520da3521e3,French Restaurant,48.849791,2.354950
1,Restaurant AT,5342fcff498e1d32177cc21b,Japanese Restaurant,48.849628,2.354654
2,Au Moulin à Vent,4d3898babb0d721ea09c55e3,French Restaurant,48.848504,2.354937
3,Grand Action,4b8a75d1f964a520a96d32e3,Indie Movie Theater,48.847614,2.352291
4,5ème Cru,4b5474d4f964a520e9bb27e3,Wine Bar,48.849066,2.354118
5,Boulangerie La Parisienne,4e58cf2bd164da2b275956ff,Bakery,48.849949,2.349760
6,Bonvivant,5530cc1f498ea6903c33a3da,Wine Bar,48.847551,2.351969
7,Anahuacalli,4adcda14f964a520fe3621e3,Mexican Restaurant,48.849907,2.350409
8,Diptyque,4adcda21f964a520b93921e3,Miscellaneous Shop,48.849589,2.351653
9,Strada Café,54842e01498ee191fa3c80e8,Café,48.847038,2.351278


In [99]:
nearby_venues.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94 entries, 0 to 93
Data columns (total 5 columns):
venue.name            94 non-null object
venue.id              94 non-null object
venue.categories      94 non-null object
venue.location.lat    94 non-null float64
venue.location.lng    94 non-null float64
dtypes: float64(2), object(3)
memory usage: 3.8+ KB


94 "venues" sont revenus dans les résultats du 13 rue de Poissy.

Juste en regardant quelques endroits sur la liste, nous pouvons voir qu'il y a pas mal de variations.

In [100]:
nearby_venues.columns = [col.split('.')[-1] for col in nearby_venues.columns]

In [101]:
nearby_venues_map = nearby_venues[0:51]

In [146]:
tileset = r'xxxxxx'
attribution = (r'Map data © <a href="http://openstreetmap.org">OpenStreetMap</a>'
                ' contributors, Imagery © <a href="http://mapbox.com">MapBox</a>')

wp_venues_map = folium.Map(location=[c_latitude, c_longitude], zoom_start=16, tiles=tileset, attr=attribution)

#red marker added for main location
folium.CircleMarker(
    [c_latitude,c_longitude],
    radius=10,
    color='red',
    popup='Adresse principale',
    fill=True,
    fill_color= 'red',
    fill_opacity = 0.6
).add_to(wp_venues_map)

#blue markers for search results
for lat, lng, label in zip(nearby_venues_map.lat, nearby_venues_map.lng, nearby_venues_map.categories):
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        color='blue',
        popup=label,
        fill=True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(wp_venues_map)
    
wp_venues_map

### Découverte de l'adresseur principal: 13 Rue de Poissy, 75005 Paris

In [105]:
nearby_venues.count()

name          94
id            94
categories    94
lat           94
lng           94
dtype: int64

Il y a 94 venues dans un rayon de 500 mètres du centre de 13 Rue de Poissy, 75005 Paris.

In [106]:
wp_unique = nearby_venues['categories'].unique()
nearby_venues['categories'].nunique()

44

Il existe 44 types de lieux différents dans le quartier.

In [107]:
top_categories = nearby_venues.groupby('categories').count()['name'].sort_values(ascending=False).head(10)
top_categories

categories
French Restaurant      22
Café                    5
Bar                     4
Bistro                  4
Hotel                   4
Pub                     4
Bakery                  3
Japanese Restaurant     3
Ice Cream Shop          3
Coffee Shop             3
Name: name, dtype: int64

La catégorie avec le plus de lieux autour sont les French Restaurants, suivie par les Cafés.

### 13 Rue de Poissy, tarifs et likes

Jetons un coup d'œil à 15 venues sites et à son prix et likes avoir pour une idée del quartier.

In [127]:
def get_venue_info(dataframe_col):
    venue_info = []
    for venue_id in dataframe_col[0:15]:
        v_url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
        v_results = requests.get(v_url).json()
        try:
            venue_name = v_results['response']['venue']['name']
        except:
            venue_name = 'Not Available'
        try:
            venue_price = v_results['response']['venue']['price']
        except:
            venue_price = 'Not Available'
        try:
            venue_likes = v_results['response']['venue']['likes']['count']
        except:
            venue_likes = 'Not Available'
        venue_info.append([venue_name, venue_price, venue_likes])
    return venue_info

In [132]:
paris_venue_info = get_venue_info(nearby_venues['id'])
print(wicker_venue_info)

[["La Tour d'Argent", {'tier': 3, 'message': 'Expensive', 'currency': '€'}, 120], ['Restaurant AT', {'tier': 2, 'message': 'Moderate', 'currency': '€'}, 50], ['Au Moulin à Vent', {'tier': 3, 'message': 'Expensive', 'currency': '€'}, 16], ['Grand Action', 'Not Available', 41], ['5ème Cru', {'tier': 2, 'message': 'Moderate', 'currency': '€'}, 27], ['Boulangerie La Parisienne', {'tier': 1, 'message': 'Cheap', 'currency': '€'}, 125], ['Bonvivant', {'tier': 2, 'message': 'Moderate', 'currency': '€'}, 41], ['Anahuacalli', {'tier': 1, 'message': 'Cheap', 'currency': '€'}, 97], ['Diptyque', 'Not Available', 25], ['Strada Café', {'tier': 1, 'message': 'Cheap', 'currency': '€'}, 287], ['Aux Merveilleux de Fred', 'Not Available', 24], ['Collège des Bernardins', 'Not Available', 34], ['Bar à Iode', {'tier': 3, 'message': 'Expensive', 'currency': '€'}, 21], ['Le Buisson Ardent', {'tier': 3, 'message': 'Expensive', 'currency': '€'}, 36], ["La Maison d'Isabelle", {'tier': 1, 'message': 'Cheap', 'curr

In [133]:
paris_df = pd.DataFrame(wicker_venue_info, columns=['name', 'price_tier', 'likes'])
paris_df

,name,price_tier,likes
0,La Tour d'Argent,"{'tier': 3, 'message': 'Expensive', 'currency'...",120
1,Restaurant AT,"{'tier': 2, 'message': 'Moderate', 'currency':...",50
2,Au Moulin à Vent,"{'tier': 3, 'message': 'Expensive', 'currency'...",16
3,Grand Action,Not Available,41
4,5ème Cru,"{'tier': 2, 'message': 'Moderate', 'currency':...",27
5,Boulangerie La Parisienne,"{'tier': 1, 'message': 'Cheap', 'currency': '€'}",125
6,Bonvivant,"{'tier': 2, 'message': 'Moderate', 'currency':...",41
7,Anahuacalli,"{'tier': 1, 'message': 'Cheap', 'currency': '€'}",97
8,Diptyque,Not Available,25
9,Strada Café,"{'tier': 1, 'message': 'Cheap', 'currency': '€'}",287


In [135]:
def get_price_tier(df):
    price_tier = df['price_tier']
    
    if len(price_tier) == 0:
        return None
    elif price_tier == 'Not Available':
        return None
    else:
        return price_tier['message']

In [136]:
paris_df['price_tier'] = wicker_df.apply(get_price_tier, axis=1)

In [137]:
paris_df.sort_values('likes', ascending=False)

,name,price_tier,likes
9,Strada Café,Cheap,287
5,Boulangerie La Parisienne,Cheap,125
0,La Tour d'Argent,Expensive,120
7,Anahuacalli,Cheap,97
1,Restaurant AT,Moderate,50
3,Grand Action,None,41
6,Bonvivant,Moderate,41
13,Le Buisson Ardent,Expensive,36
11,Collège des Bernardins,None,34
14,La Maison d'Isabelle,Cheap,29


De ces 15 "venues", il semblerait que le prix moyen d'un magasin autour du 13 rue de Poissy soit dans la fourchette modérée-bon marché. Sur les 15, quatre seulement sont considérés comme chers. Le nombre de «Likes» pour ces lieux n'est pas trop faible, ce qui suggère qu'ils ne sont pas mal notés non plus.

###  Importer les données de localisation de Lyon depuis l'API Foursquare

In [13]:
qa_address = '6 Rue Gasparin, 69002 Lyon, France'

qa_geolocator = Nominatim(user_agent='foursquare_agent')
qa_location = qa_geolocator.geocode(qa_address)
qa_lat = qa_location.latitude
qa_lng = qa_location.longitude

print('6 Rue Gasparin, 69002 Lyon, France: ', qa_lat, qa_lng)

6 Rue Gasparin, 69002 Lyon, France:  45.7598522 4.8333555


In [14]:
qa_url = 'xxxxxx'.format(
        CLIENT_ID,
        CLIENT_SECRET,
        qa_lat,
        qa_lng,
        VERSION,
        radius,
        LIMIT
        )

print(qa_url)

xxxxxx


In [141]:
qa_results = requests.get(qa_url).json()

In [142]:
qa_venues = qa_results['response']['groups'][0]['items']

qa_nearby_venues = json_normalize(qa_venues)

qa_nearby_venues = qa_nearby_venues.loc[:, filtered_columns]

In [143]:
qa_nearby_venues['venue.categories'] = qa_nearby_venues.apply(get_category_type, axis=1)
qa_nearby_venues.columns = [col.split('.')[-1] for col in qa_nearby_venues.columns]

qa_nearby_venues.head(10)

,name,id,categories,lat,lng
0,Place des Jacobins,4b61baf8f964a520361f2ae3,Plaza,45.760489,4.833636
1,Place des Célestins,4b61bb38f964a5203f1f2ae3,Plaza,45.759804,4.831842
2,Place de la République,4cbfe494021ca1438bf15e0c,Plaza,45.760347,4.835635
3,Slake Coffee House,5602838f498e9111e2522df3,Café,45.760922,4.832910
4,Marché Saint-Antoine,4c304d6a66e40f470f65c38b,Farmers Market,45.761756,4.831853
5,Mozzato - Comptoir à Mozzarella,526038cd11d2d0efdb9f407f,Italian Restaurant,45.761697,4.832963
6,Place Bellecour,4b5e0a3df964a520817a29e3,Plaza,45.757523,4.832397
7,Théâtre des Célestins,4b8a84caf964a520567032e3,Theater,45.759769,4.831996
8,L'Épicerie - Bistrot à Tartines,4b6ef616f964a52068d32ce3,Restaurant,45.761931,4.832668
9,Pignol Patissier & Restaurant,5066b98416482fabcf2082b4,Dessert Shop,45.758589,4.832523


In [144]:
qa_nearby_venues.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 5 columns):
name          65 non-null object
id            65 non-null object
categories    65 non-null object
lat           65 non-null float64
lng           65 non-null float64
dtypes: float64(2), object(3)
memory usage: 2.6+ KB


65 "venues" sont revenus dans les résultats du Rue Gasparin, 69002 Lyon.

In [204]:

qa_venues_map = folium.Map(location=[qa_lat, qa_lng], zoom_start=16, tiles=tileset, attr=attribution)


folium.CircleMarker(
    [qa_lat,qa_lng],
    radius=10,
    color='red',
    popup='Rue Gasparin',
    fill=True,
    fill_color= 'red',
    fill_opacity = 0.6
).add_to(qa_venues_map)


for lat, lng, label in zip(qa_nearby_venues.lat, qa_nearby_venues.lng, qa_nearby_venues.categories):
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        color='blue',
        popup=label,
        fill=True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(qa_venues_map)
    
qa_venues_map

### Explorer le quartier

In [149]:
qa_unique = qa_nearby_venues['categories'].unique()
qa_nearby_venues['categories'].nunique()

37

 Rue Gasparin dispose de 39 types de "venues" différents.

In [150]:
qa_wp_same = []
qa_has = []
for x in qa_unique:
    if x in wp_unique:
        qa_wp_same.append(x)
    else:
        qa_has.append(x)
print('Emplacement en Lyon et Emplacement en Paris: ', '\n', qa_wp_same)
print('\n')
print('Emplacement en Lyon: ', '\n', qa_has)

wp_has = []
for x in wp_unique:
    if x in qa_unique:
        continue
    else:
        wp_has.append(x)
print('\n')
print('Emplacement en Paris: ', '\n', wp_has)

Emplacement en Lyon et Emplacement en Paris:  
 ['Plaza', 'Café', 'Italian Restaurant', 'Hotel', 'Pub', 'French Restaurant', 'Bar', 'Coffee Shop', 'Seafood Restaurant', 'Ice Cream Shop', 'Historic Site', 'Cupcake Shop', 'Lebanese Restaurant', 'Bookstore', 'Museum', 'Japanese Restaurant', 'Taco Place']


Emplacement en Lyon:  
 ['Farmers Market', 'Theater', 'Restaurant', 'Dessert Shop', 'Bagel Shop', 'Sushi Restaurant', 'Clothing Store', 'Resort', 'Gastropub', 'Donut Shop', 'Burger Joint', 'Theme Park', 'Department Store', 'Road', 'Chocolate Shop', 'Church', 'Beer Bar', 'Comedy Club', 'American Restaurant', 'Tea Room']


Emplacement en Paris:  
 ['Indie Movie Theater', 'Wine Bar', 'Bakery', 'Mexican Restaurant', 'Miscellaneous Shop', 'Pastry Shop', 'Diner', 'Bistro', 'Cheese Shop', 'Trail', 'Pool', 'Basque Restaurant', 'Hobby Shop', 'Portuguese Restaurant', 'Ethiopian Restaurant', 'Liquor Store', 'Wine Shop', 'Tapas Restaurant', 'Park', 'Noodle House', 'Breakfast Spot', 'Creperie', 'Veg

Ci-dessus, nous pouvons voir qu'il y a beaucoup plus de types de "venues" différents dans le quartier en Paris que dans Lyon. Cela suggère qu'il y a plus de variété à Paris qu'à Lyon.

In [151]:
qa_nearby_venues.groupby('categories').count()['name'].sort_values(ascending=False).head(10)

categories
French Restaurant     7
Plaza                 6
Café                  5
Hotel                 4
Italian Restaurant    3
Clothing Store        2
Bar                   2
Sushi Restaurant      2
Burger Joint          2
Resort                2
Name: name, dtype: int64

Les catégories de "venues" sont énumérées la plus fréquente à la liste. Nous listons ici le top 10.

###  Rue Gasparin, 69002 Lyon, tarifs et likes

Jetons un coup d'œil à 15 venues sites et à son prix et likes avoir pour une idée del quartier.

In [152]:
qa_venue_info = get_venue_info(qa_nearby_venues['id'])

In [153]:
qa_venue_info

[['Place des Jacobins', 'Not Available', 198],
 ['Place des Célestins', 'Not Available', 38],
 ['Place de la République', 'Not Available', 127],
 ['Slake Coffee House', {'tier': 1, 'message': 'Cheap', 'currency': '€'}, 148],
 ['Marché Saint-Antoine', 'Not Available', 77],
 ['Mozzato - Comptoir à Mozzarella',
  {'tier': 2, 'message': 'Moderate', 'currency': '€'},
  64],
 ['Place Bellecour', 'Not Available', 516],
 ['Théâtre des Célestins', 'Not Available', 26],
 ["L'Épicerie - Bistrot à Tartines",
  {'tier': 2, 'message': 'Moderate', 'currency': '€'},
  185],
 ['Pignol Patissier & Restaurant',
  {'tier': 1, 'message': 'Cheap', 'currency': '€'},
  23],
 ['Mercure Lyon Centre Beaux-Arts', 'Not Available', 44],
 ['Best Bagels Company',
  {'tier': 2, 'message': 'Moderate', 'currency': '€'},
  74],
 ['MOS - Maker of Stories',
  {'tier': 1, 'message': 'Cheap', 'currency': '€'},
  6],
 ['Gangnam', {'tier': 2, 'message': 'Moderate', 'currency': '€'}, 10],
 ['La Voûte Chez Léa',
  {'tier': 3, 'm

In [154]:
qa_df = pd.DataFrame(qa_venue_info, columns=['name', 'price_tier', 'likes'])
qa_df    

,name,price_tier,likes
0,Place des Jacobins,Not Available,198
1,Place des Célestins,Not Available,38
2,Place de la République,Not Available,127
3,Slake Coffee House,"{'tier': 1, 'message': 'Cheap', 'currency': '€'}",148
4,Marché Saint-Antoine,Not Available,77
5,Mozzato - Comptoir à Mozzarella,"{'tier': 2, 'message': 'Moderate', 'currency':...",64
6,Place Bellecour,Not Available,516
7,Théâtre des Célestins,Not Available,26
8,L'Épicerie - Bistrot à Tartines,"{'tier': 2, 'message': 'Moderate', 'currency':...",185
9,Pignol Patissier & Restaurant,"{'tier': 1, 'message': 'Cheap', 'currency': '€'}",23


In [155]:
qa_df['price_tier'] = qa_df.apply(get_price_tier, axis=1)
qa_df.sort_values('likes', ascending=False)

,name,price_tier,likes
6,Place Bellecour,None,516
0,Place des Jacobins,None,198
8,L'Épicerie - Bistrot à Tartines,Moderate,185
3,Slake Coffee House,Cheap,148
2,Place de la République,None,127
4,Marché Saint-Antoine,None,77
11,Best Bagels Company,Moderate,74
5,Mozzato - Comptoir à Mozzarella,Moderate,64
10,Mercure Lyon Centre Beaux-Arts,None,44
1,Place des Célestins,None,38


Il semble que ce quartier de Lyon soit plus attractif pour les touristes, cependant il a des locaux plus chers par rapport à Paris

### Importer les données de localisation de Marseille depuis l'API Foursquare

In [11]:
address = '2 Rue du Chevalier Roze 13002 Marseille, France'

ch_geolocator = Nominatim(user_agent='foursquare_agent')
ch_location = ch_geolocator.geocode(address)
ch_lat = ch_location.latitude
ch_lng = ch_location.longitude

print('2 Rue du Chevalier Roze 13002 Marseille, France: ', ch_lat, ch_lng)

2 Rue du Chevalier Roze 13002 Marseille, France:  43.2973801 5.3724761


In [12]:
ch_url = 'xxxxxx'.format(
        CLIENT_ID,
        CLIENT_SECRET,
        ch_lat,
        ch_lng,
        VERSION,
        radius,
        LIMIT
        )

print(ch_url)

xxxxxx


In [192]:
ch_results = requests.get(ch_url).json()

In [193]:
ch_venues = ch_results['response']['groups'][0]['items']

ch_nearby_venues = json_normalize(ch_venues)

ch_nearby_venues = ch_nearby_venues.loc[:,filtered_columns]

ch_nearby_venues.head()

,venue.name,venue.id,venue.categories,venue.location.lat,venue.location.lng
0,La Caravelle,4b5b300ef964a520b1e928e3,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",43.296535,5.372060
1,Starbucks (Starbucks Coffee),4bea370a61aca593e7908300,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",43.297847,5.372830
2,InterContinental Marseille Hôtel-Dieu,50d1943ae4b0fff6c391682b,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",43.298661,5.369779
3,Les Buvards,4bc57ecfccbcef3b1399e6d2,"[{'id': '4bf58dd8d48988d123941735', 'name': 'W...",43.297783,5.371368
4,Galeries Lafayette,4bbdf9ce078095218635db91,"[{'id': '4bf58dd8d48988d1f6941735', 'name': 'D...",43.296544,5.374758


In [194]:
ch_nearby_venues['venue.categories'] = ch_nearby_venues.apply(get_category_type,axis = 1)
ch_nearby_venues.columns = [col.split('.')[-1] for col in ch_nearby_venues.columns]
ch_nearby_venues.head(10)

,name,id,categories,lat,lng
0,La Caravelle,4b5b300ef964a520b1e928e3,Bar,43.296535,5.372060
1,Starbucks (Starbucks Coffee),4bea370a61aca593e7908300,Coffee Shop,43.297847,5.372830
2,InterContinental Marseille Hôtel-Dieu,50d1943ae4b0fff6c391682b,Hotel,43.298661,5.369779
3,Les Buvards,4bc57ecfccbcef3b1399e6d2,Wine Bar,43.297783,5.371368
4,Galeries Lafayette,4bbdf9ce078095218635db91,Department Store,43.296544,5.374758
5,Terrasse de l'InterContinental Marseille Hôtel...,51bc60145019f457c9e71423,Roof Deck,43.298463,5.369761
6,Capian Bar,51ad0479498eafba19f59d33,Cocktail Bar,43.298360,5.369721
7,Spa de l'InterContinental Marseille Hôtel-Dieu,51bc5f505019f457c9e67182,Spa,43.298510,5.369782
8,Hôtel la Résidence du Vieux-Port,4bd0b48ba8b3a593b770645f,Hotel,43.296483,5.372727
9,Vanille Noire,53f87ce2498ecc0a79b29359,Ice Cream Shop,43.297199,5.368588


In [195]:
ch_nearby_venues.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83 entries, 0 to 82
Data columns (total 5 columns):
name          83 non-null object
id            83 non-null object
categories    83 non-null object
lat           83 non-null float64
lng           83 non-null float64
dtypes: float64(2), object(3)
memory usage: 3.3+ KB


83 "venues" sont revenus dans les résultats du 2 Rue du Chevalier Roze 13002 Marseille.

In [196]:
ch_nearby_venues_map = ch_nearby_venues[0:10]

In [205]:
ch_venues_map = folium.Map(location=[ch_lat, ch_lng], zoom_start=17, tiles=tileset, attr=attribution)

#red marker added for main location
folium.CircleMarker(
    [ch_lat,ch_lng],
    radius=10,
    color='red',
    popup='2 Rue du Chevalier Roze 13002 Marseille, France',
    fill=True,
    fill_color= 'red',
    fill_opacity = 0.6
).add_to(ch_venues_map)

#blue markers for search results
for lat, lng, label in zip(ch_nearby_venues_map.lat, ch_nearby_venues_map.lng, ch_nearby_venues_map.categories):
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        color='blue',
        popup=label,
        fill=True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(ch_venues_map)
    
ch_venues_map

### Explorer le quartier

In [199]:
ch_unique = ch_nearby_venues['categories'].unique()
ch_nearby_venues['categories'].nunique()

46

Il y a 46 types uniques de lieux autour du Rue Lulli 13001 Marseille.

In [200]:
ch_wp_same = []
ch_has = []
for x in ch_unique:
    if x in wp_unique:
        ch_wp_same.append(x)
    else:
        ch_has.append(x)
print('Emplacement en Marseille et Emplacement en Paris: ', '\n', ch_wp_same)
print('\n')
print('Emplacement en Marseille: ', '\n', ch_has)

wp_has = []
for x in wp_unique:
    if x in ch_unique:
        continue
    else:
        wp_has.append(x)
print('\n')
print('Emplacement en Paris: ', '\n', wp_has)

Emplacement en Marseille et Emplacement en Paris:  
 ['Bar', 'Coffee Shop', 'Hotel', 'Wine Bar', 'Ice Cream Shop', 'French Restaurant', 'Café', 'Plaza', 'Museum', 'Pub', 'Seafood Restaurant', 'Bistro', 'Italian Restaurant', 'Vegetarian / Vegan Restaurant', 'Bookstore', 'Trail', 'Diner']


Emplacement en Marseille:  
 ['Department Store', 'Roof Deck', 'Cocktail Bar', 'Spa', 'Tea Room', 'Provençal Restaurant', 'Food', 'Lounge', 'Public Art', 'Pizza Place', 'Moroccan Restaurant', 'Garden', 'Candy Store', 'Burger Joint', 'Fish Market', 'History Museum', 'Electronics Store', 'Cosmetics Shop', 'Sandwich Place', 'Sporting Goods Shop', 'Steakhouse', 'Tourist Information Center', 'Fast Food Restaurant', 'Shopping Mall', 'Supermarket', 'Speakeasy', 'Watch Shop', 'Metro Station', 'Convenience Store']


Emplacement en Paris:  
 ['Japanese Restaurant', 'Indie Movie Theater', 'Bakery', 'Mexican Restaurant', 'Miscellaneous Shop', 'Pastry Shop', 'Historic Site', 'Cheese Shop', 'Pool', 'Basque Restaura

Marseille a similar variété de "venues" que Paris.

In [206]:
ch_nearby_venues.groupby('categories').count()['name'].sort_values(ascending=False).head(10)

categories
Hotel                  10
French Restaurant       7
Bar                     3
Ice Cream Shop          3
Italian Restaurant      3
Moroccan Restaurant     3
Plaza                   3
Electronics Store       3
Coffee Shop             3
Seafood Restaurant      3
Name: name, dtype: int64

### 2 Rue du Chevalier Roze 13002 Marseille, tarifs et likes

Jetons un coup d'œil aux informations sur du quartier

In [201]:
ch_venue_info = get_venue_info(ch_nearby_venues['id'])

In [202]:
ch_venue_info

[['La Caravelle', {'tier': 2, 'message': 'Moderate', 'currency': '€'}, 96],
 ['Starbucks (Starbucks Coffee)',
  {'tier': 3, 'message': 'Expensive', 'currency': '€'},
  147],
 ['InterContinental Marseille Hôtel-Dieu', 'Not Available', 126],
 ['Les Buvards', {'tier': 2, 'message': 'Moderate', 'currency': '€'}, 17],
 ['Galeries Lafayette', 'Not Available', 101],
 ["Terrasse de l'InterContinental Marseille Hôtel-Dieu", 'Not Available', 27],
 ['Capian Bar', {'tier': 3, 'message': 'Expensive', 'currency': '€'}, 12],
 ["Spa de l'InterContinental Marseille Hôtel-Dieu", 'Not Available', 12],
 ['Hôtel la Résidence du Vieux-Port', 'Not Available', 14],
 ['Vanille Noire', {'tier': 1, 'message': 'Cheap', 'currency': '€'}, 21],
 ['La Fabrique', {'tier': 3, 'message': 'Expensive', 'currency': '€'}, 20],
 ['Cup of Tea', {'tier': 2, 'message': 'Moderate', 'currency': '€'}, 19],
 ['Au Cœur du Panier', 'Not Available', 51],
 ['Amorino', {'tier': 1, 'message': 'Cheap', 'currency': '€'}, 69],
 ['Une Table 

In [203]:
ch_df = pd.DataFrame(ch_venue_info, columns=['name', 'price_tier', 'likes'])
ch_df['price_tier'] = ch_df.apply(get_price_tier, axis=1)
ch_df.sort_values('likes', ascending=False)

,name,price_tier,likes
1,Starbucks (Starbucks Coffee),Expensive,147
2,InterContinental Marseille Hôtel-Dieu,None,126
4,Galeries Lafayette,None,101
0,La Caravelle,Moderate,96
13,Amorino,Cheap,69
12,Au Cœur du Panier,None,51
5,Terrasse de l'InterContinental Marseille Hôtel...,None,27
9,Vanille Noire,Cheap,21
10,La Fabrique,Expensive,20
11,Cup of Tea,Moderate,19


On dirait que Lyon a des restaurants beaucoup plus chers.

# Résultats

##### Échelle des prix

Sur la base de l'analyse, il semblerait que Marseille ait une gamme de prix plus proche de Paris que de Lyon. Cependant, en raison du nombre limité d'appels premium par jour, il est possible que les restaurants chers à Lyon aient simplement été manqués. Lyon avait également plus de sites qui n'ont pas de niveau de prix étiqueté, ce qui rend également la comparaison plus difficile. aussi les nouveaux sites (lyon et marseille) ont moins de restaurants que Paris.

##### Popularity

Les lieux de Lyon ont beaucoup plus de likes que Paris ou Marseille. Cela signifie que Lyon pourrait être une zone plus fréquentée avec plus de trafic piétonnier. Ou les gens sont très enthousiaste dans ce domaine.

# Discussion

Après l'anaylsis, il semblerait que Lyon soit le meilleur choix pour quelqu'un qui cherche une expérience plus similaire à Paris. C'est un peu plus cher, mais cela fournira également un style de vie divertissant.

Marseille semble plus calme et tranquille. Ce serait un bon choix pour quelqu'un qui voudrait changer ce qu'il a obtenu à Paris.

# Références

<a href="https://es.coursera.org/learn/applied-data-science-capstone"> The Battle of Neighborhoods: Coursera Capstone Project</a>

<a href="https://pypi.org/project/geopy/"> Geopy Geocoders</a>

<a href="https://developer.foursquare.com/"> Foursquare API</a>

<a href="https://github.com/mapbox/mapboxgl-jupyter"> MapBox Location Data Visualization library for Jupyter Notebooks</a>

## 👍👍 <center> <a href="https://www.linkedin.com/in/lealrlx/">Partagez vos idées et vos commentaires, écrivez-moi 🌍 </a> </center>##